#### Часть 1.

Собственная имплементация

In [ ]:
import numpy as np

Прочитаем текст:

In [ ]:
with open('zbch.txt') as f:
    data = f.read()
    
data

Разделим на слова:

In [ ]:
!pip install razdel

In [ ]:
from razdel import tokenize

ind_words = [t.text for t in tokenize(data)]

Напишем функцию, которая будет нам собирать наши пары слов, и с ее помощью ниже получим словарь с переходами:

In [ ]:
def make_pairs(ind_words):
    for i in range(len(ind_words) - 1):
        yield ind_words[i], ind_words[i + 1]
        
pair = make_pairs(ind_words)

In [ ]:
word_dict = {}

for word_1, word_2 in pair:
    if word_1 in word_dict.keys():
        word_dict[word_1].append(word_2)
    else:
        word_dict[word_1] = [word_2]
        
word_dict

Рандомно выберем первое слово и начнем генерировать текст длиной в 20 слов, случайно выбирая возможные следующие токены по предыдущему. 

In [ ]:
first_word = np.random.choice(ind_words)
chain = [first_word]
n_words = 20

for i in range(n_words):
    chain.append(np.random.choice(word_dict[chain[-1]]))

Наш обучающий текст был очень маленьким, поэтому, скорее всего, сгенерируются его же фрагменты вперемешку. На больших текстах результат будет интереснее.

In [ ]:
print(' '.join(chain))

#### Часть 2

Теперь будем использовать готовую библиотеку markovify. Для предобработки текстов (возьмем Шекспира) нам понадобятся nltk и spacy.

In [ ]:
!pip install nltk
!pip install spacy
!pip install markovify
!python -m spacy download en

Загрузим гутенберговский корпус из nltk и возьмем три пьесы Шекспира из него:

In [ ]:
import spacy
import re
import markovify
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings('ignore')

nltk.download('gutenberg')
print(gutenberg.fileids())

In [ ]:
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
caesar = gutenberg.raw('shakespeare-caesar.txt')
# выведем по 100 первых символов
print('\nRaw:\n', hamlet[:100])
print('\nRaw:\n', macbeth[:100])
print('\nRaw:\n', caesar[:100])

Напишем всякие функции предобработки:

In [ ]:
# поудаляем ненужные пробелы, отступы, пунктуацию и прочее
def text_cleaner(text):
    text = re.sub(r'--', ' ', text)
    text = re.sub('[\[].*?[\]]', '', text)
    text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
    text = ' '.join(text.split())
    return text

In [ ]:
# удалим заголовки глав
hamlet = re.sub(r'Chapter \d+', '', hamlet)
macbeth = re.sub(r'Chapter \d+', '', macbeth)
caesar = re.sub(r'Chapter \d+', '', caesar)
# применим функции к текстам
hamlet = text_cleaner(hamlet)
caesar = text_cleaner(caesar)
macbeth = text_cleaner(macbeth)

In [ ]:
# распарсим с помощью спейси: это даст нам токены, предложения и потом части речи
nlp = spacy.load('en_core_web_sm')
hamlet_doc = nlp(hamlet)
macbeth_doc = nlp(macbeth)
caesar_doc = nlp(caesar)

Соберем предложения, отсеяв однословные

In [ ]:
hamlet_sents = ' '.join([sent.text for sent in hamlet_doc.sents if len(sent.text) > 1])
macbeth_sents = ' '.join([sent.text for sent in macbeth_doc.sents if len(sent.text) > 1])
caesar_sents = ' '.join([sent.text for sent in caesar_doc.sents if len(sent.text) > 1])

In [ ]:
shakespeare_sents = hamlet_sents + macbeth_sents + caesar_sents

In [ ]:
print(shakespeare_sents[:3])

Теперь самое интересное: инициализируем наш markovify предложениями из текстов. Внутри этот объект уже сам сделает все, что нам надо, и нам останется только генерировать "предложения".

In [ ]:
generator_1 = markovify.Text(shakespeare_sents, state_size=3)

In [ ]:
# попробуем сгенерировать три предложения без всяких ограничений
for i in range(3):
    print(generator_1.make_sentence())

# сгенерируем предложения с ограничением: не больше 100 символов
for i in range(3):
    print(generator_1.make_short_sentence(max_chars=100))

Прикольно, но не совсем. Давайте для улучшения нашего генератора используем информацию о частях речи: мы ее просто приклеим к нашим токенам, переопределив пару методов в классе Text, так, чтобы при разбиении предложений в исходных данных он приклеивал части речи и использовал их, собирая матрицу переходов, а при создании генерированных предложений обратно отклеивал.

In [ ]:
class POSifiedText(markovify.Text):

    def word_split(self, sentence):
        return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = ' '.join(word.split('::')[0] for word in words)
        return sentence

# инициализируем экземпляр доопределенного класса
generator_2 = POSifiedText(shakespeare_sents, state_size=3)

А теперь потестим.

In [ ]:
for i in range(5):
    print(generator_2.make_sentence())

for i in range(5):
    print(generator_2.make_short_sentence(max_chars=100))